In [163]:
from scipy.linalg import sqrtm
import cvxpy as cp
from sympy.utilities.lambdify import lambdify
import sympy as sp
import numpy as np
from scipy.linalg import solve_discrete_are as dare 
from scipy.signal import cont2discrete
import time
from matplotlib.pyplot import *


Coupled masses dynamical system:

$$
\begin{cases}
m_1\ddot{y}_1 + k(y_1 - y_2) = F \\ 
m_2\ddot{y}_2 + k(y_2 - y_1) = 0
\end{cases}
$$

In [164]:
# system parameters
m_1 = 1
m_2 = 3
K = 100

param = m_1, m_2, K
u_d = 0

x_d = np.array([0.1,0.1,0,0])
Q = 10*np.diag([1,100,0.1,1])
print(Q[2:4, 2:4])
# x_d = np.array([0.1,0,0.1,0])
# Q = 10*np.diag([1,0.1,100,1])

R = np.array([10])
C = np.array([[1, 0, 0 ,0]])
D = np.array([[0, 0]])
# N = 200
t0 = 0
T = 5
freq = 100
N = int(freq*T)
dt = 1/freq
t = np.array(range(N+1))*dt

[[ 1.  0.]
 [ 0. 10.]]


In [165]:
m1, m2, k = sp.symbols(r'm1 m2 k')

x_sym = sp.symbols(r'\y1 y2 \dot{\y1} \dot{y2}')
u_sym = sp.symbols(r'u')

def f_cp(x, u, parameters):
    m1, m2, k = parameters
    y1, dy1, y2, dy2 = x

    ddy1 = 1/m1 *(u - k*(y1 - y2))
    ddy2 = 1/m2 *(-k*(y2 - y1))

    return dy1, dy2, ddy1, ddy2 

f_sym = sp.Matrix(f_cp(x_sym, u_sym, (m1, m2, k)))
Jx_sym = f_sym.jacobian([x_sym])
Ju_sym = f_sym.jacobian([u_sym])
Jx_sym
    

Matrix([
[    0, 1,     0, 0],
[    0, 0,     0, 1],
[-k/m1, 0,  k/m1, 0],
[ k/m2, 0, -k/m2, 0]])

In [166]:
Ju_sym

Matrix([
[   0],
[   0],
[1/m1],
[   0]])

In [167]:
A_fnc = lambdify([x_sym, u_sym, [m1, m2, k]], Jx_sym)
B_fnc = lambdify([x_sym, u_sym, [m1, m2, k]], Ju_sym)

In [168]:
A = A_fnc(x_d, u_d, [m_1, m_2, K])
B = B_fnc(x_d, u_d, [m_1, m_2, K])

In [169]:
A_d, B_d, C_d, D_d, _ = cont2discrete((A,B,C,D), dt)

def dlqr(A, B, Q, R):
    '''Discrete time LTI LQR'''
    # Solve the DARE
    P = dare(A, B, Q, R)
    # Compute the LQR gain
    R_inv = np.linalg.inv(R + B.T @ P @ B )
    K = R_inv @ (B.T @ P @ A)
    return K, P

K_LQR, P = dlqr(A_d, B_d, Q, R)
print(P.shape)
P
# P = P[0:2, 0:2]
# P

(4, 4)


array([[ 233735.21929428,   21919.18500926, -221135.35653536,
          12418.85962366],
       [  21919.18500926,   82562.02934609,  -13665.54729739,
          30463.4221373 ],
       [-221135.35653536,  -13665.54729739,  219910.1640723 ,
          -9174.53210694],
       [  12418.85962366,   30463.4221373 ,   -9174.53210694,
          23914.94869022]])

In [170]:
M_mat = np.array([[m_1, 0], [0, m_2]])

def V_fun(x, param):
    m1, m2, k = param
    y1, y2 = x
    V1 = k*(y1 - y2)
    V2 = k*(y2 - y1)
    out = cp.bmat((V1,
            V2))
    return out

In [171]:
# define dimensions
H, n, m = 20, 2, 1

# define variables
U = cp.Variable((m, H), name='U')
X = cp.Variable((n, H+1), name='X')

# X1 = cp.Variable((1, H+1), name='X1')
# X2 = cp.Variable((1, H+1), name = 'X2')

# define parameters
Psqrt = sqrtm(P)
Qsqrt = sqrtm(Q)
Rsqrt = sqrtm([R])
U_max = cp.Parameter(m, name='U_max')
U_min = cp.Parameter(m, name='U_min')
X_max = cp.Parameter((n,1), name='X_max')
X_min = cp.Parameter((n,1), name='X_min')

x_init_0 = cp.Parameter(n, name='x_init_0')
x_init_1 = cp.Parameter(n, name='x_init_1')
# q = cp.Parameter((4, H+1), name='q')
# VEL = cp.Parameter((n, H+1), name = 'VEL')

VEL = (X[:,1:] - X[:,:H])/dt 
q = cp.vstack((X[:,:H], VEL))

# VEL1 = cp.Parameter((1, H+1), name='VEL1')
# VEL2 = cp.Parameter((1, H+1), name='VEL2')
# X = cp.Parameter((n, H+1), name = 'X')
# X = cp.vstack((X1, X2))

# VEL1 = (X1[:,1:] - X1[:,:H])/dt 
# VEL2 = (X2[:,1:] - X2[:,:H])/dt 

# q = cp.vstack((X1[:,:H], VEL1, X2[:,:H], VEL2))
# print((Psqrt@q).shape)
# print((Qsqrt@q).shape)

# define objective
objective = cp.Minimize(cp.sum_squares(Psqrt@q[:,H-1]) + cp.sum_squares(Qsqrt@q[:,:H])
                          + cp.sum_squares(Rsqrt@U)) 

# objective = cp.Minimize(cp.sum_squares(Psqrt[0:2, 0:2]@ X[:,:H]) + cp.sum_squares(Qsqrt[0:2, 0:2]@X[:,:H])
#                         + cp.sum_squares(Psqrt[2:4, 2:4]@VEL) + cp.sum_squares(Qsqrt[2:4, 2:4]@VEL)
#                           + cp.sum_squares(Rsqrt@U))                           

deriv = (X[:,2:] - 2*X[:,1:-1] + X[:,:H-1])/(dt**2)
dyn = M_mat @ deriv + V_fun(X[:,1:-1], param) - (np.array([[1], [0]])@ U[:,:-1])

# define constraints
constraints = [dyn == 0,
               U <= U_max,
               U >= U_min,
               X <= X_max,
               X >= X_min,
            #    X[:,H] == np.zeros(2), 
               X[:,0] == x_init_0,
               X[:,1] == x_init_1]

# define problem
problem = cp.Problem(objective, constraints)

In [172]:
x0 = np.array([0.2, 0.2])
x1 = np.array([0.2, 0.2])

problem.param_dict['U_max'].value = [10]
problem.param_dict['U_min'].value = [-10]
problem.param_dict['X_max'].value = 1000*np.ones((2,1))
problem.param_dict['X_min'].value = -1000*np.ones((2,1))
problem.param_dict['x_init_0'].value = x0
problem.param_dict['x_init_1'].value = x1

t0 = time.time()
val = problem.solve()
t1 = time.time()
print('\nCVXPY\nSolve time: %.3f ms' % (1000 * (t1 - t0)))
print('Objective function value: %.6f\n' % val)

X_opt = problem.var_dict['X'].value
U_opt = problem.var_dict['U'].value

In [173]:
t0 = time.time()
val = problem.solve()
t1 = time.time()
print('\nCVXPY\nSolve time: %.3f ms' % (1000 * (t1 - t0)))
print('Objective function value: %.6f\n' % val)

X_opt = problem.var_dict['X'].value
# X2_opt = problem.var_dict['X2'].value

U_opt = problem.var_dict['U'].value


CVXPY
Solve time: 44.010 ms
Objective function value: inf



In [174]:
print(X_opt.shape)
y1, y2 = X_opt

plot(y1, 'r', linewidth=2.0, label = r'$y1$')
plot(y2, 'b', linewidth=2.0, label = r'$y2$')
grid(color='black', linestyle='--', linewidth=1.0, alpha = 0.7)
grid(True)
legend()
ylabel('distance')
xlabel('Time')
show()

plot(U_opt[0,:], 'r', linewidth=2.0, label = "U_opt")
grid(color='black', linestyle='--', linewidth=1.0, alpha = 0.7)
grid(True)
legend()
ylabel('control')
xlabel('Time')
show()

AttributeError: 'NoneType' object has no attribute 'shape'

### Simulation

In [ ]:
# u_max = 100
# solve_time = []
# U = 0
# X = x0

# problem.param_dict['X_max'].value = np.array([[100], [100]])
# problem.param_dict['X_min'].value = np.array([[-100], [-100]])
# problem.param_dict['U_max'].value = [u_max]
# problem.param_dict['U_min'].value = [-u_max]

# for k in range(N):
#   # MPC CONTROL
#   problem.param_dict['x_init'].value = x0
#   val = problem.solve()

#   U_opt = problem.var_dict['U'].value

#   u = U_opt[0][0]

#   # x0 = A_d@x0 + B_d@[u]
  
#   # deriv = (X[:,2:] - 2*X[:,1:-1] + X[:,:H-1])/(dt**2)

#   # dyn = M_mat @ deriv + V_fun(X[:,1:-1], param) - (np.array([[1], [0]])@ U[:,:-1])

  
#   X = np.vstack((X, x0))
#   U = np.vstack((U, u))

# y1, y2 = np.split(X, 2, axis = 1)
# t = np.array(range(N+1))*dt


In [ ]:
# plot(t, y1, 'r', linewidth=2.0, label = r'$y1$')
# plot(t, y2, 'b', linewidth=2.0, label = r'$y2$')
# grid(color='black', linestyle='--', linewidth=1.0, alpha = 0.7)
# grid(True)
# legend()
# ylabel(r'Distance $y1, y2$')
# xlabel(r'Time $t$ (s)')
# show()

In [ ]:
# plot(t, U, 'r', linewidth=2.0, label = "U_opt")
# grid(color='black', linestyle='--', linewidth=1.0, alpha = 0.7)
# grid(True)
# legend()
# ylabel('control')
# xlabel('Time')
# show()